In [1]:
puzzle <- readFile "16.txt"
plines = lines puzzle

# Day 16

### Puzzle 1

In [2]:
import Data.List.Split

type Rule = (String, [(Int, Int)])
type Ticket = [Int]

parseRule :: String -> Rule
parseRule line = (name, map ((\[a, b] -> (a, b)) . map read . splitOn "-") $ splitOn " or " ranges)
    where [name, ranges] = splitOn ": " line

rules = map parseRule . lines . head . splitOn "\n\n" $ puzzle :: [Rule]

nearbyTickets = map (map read . splitOn ",") . tail . lines . last . splitOn "\n\n" $ puzzle :: [Ticket]

checkRule :: Int -> Rule -> Bool
checkRule num = any (\(a, b) -> a <= num && num <= b) . snd

fieldValid :: Int -> Bool
fieldValid field = any (checkRule field) rules

sum [field | ticket <- nearbyTickets, field <- ticket, not $ fieldValid field]

25788

### Puzzle 2

In [3]:
import qualified Data.Set as Set
import qualified Data.Map as Map
import Data.List (isPrefixOf)
import Data.Maybe

n = length rules :: Int

myTicket = map read . splitOn "," . last . lines . (!!1) . splitOn "\n\n" $ puzzle :: Ticket

validTickets = filter (all fieldValid) nearbyTickets :: [Ticket]

possibleAllocations = [Set.fromList $ filter (possible rule) [0..n-1] | rule <- [0..n-1]] :: [Set.Set Int]
    where possible rule field = all (\ticket -> checkRule (ticket !! field) (rules !! rule)) validTickets

clearPossibility :: [Set.Set Int] -> Int -> [Set.Set Int]
clearPossibility possible num = map (Set.delete num) possible

reduce :: [Maybe Int] -> [Set.Set Int] -> [Int]
reduce allocations possible = solve newAllocations newPossibleAllocations
    where newAllocations = [case Set.toList $ possible !! i of 
                                [x] -> Just x 
                                _ -> y | (i, y) <- zip [0..n-1] allocations]
          newPossibleAllocations = foldl clearPossibility possible $ catMaybes newAllocations

solve :: [Maybe Int] -> [Set.Set Int] -> [Int]
solve allocations possible
    | not $ any isNothing allocations = catMaybes allocations
    | otherwise = reduce allocations possible

departureValues :: [Int] -> Int
departureValues allocation = product $ map ((myTicket!!) . (allocation!!)) departureRules
    where departureRules = [i | (i, (name, _)) <- zip [0..n-1] rules, "departure" `isPrefixOf` name]

departureValues $ solve (replicate n Nothing) possibleAllocations

3902565915559